In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables
load_dotenv()

MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DB")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

In [3]:
#Connect to the database
connection_string = f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"
engine = create_engine(connection_string)

In [4]:
# Load data from MySQL table
query = "SELECT * FROM network_logs;"
df = pd.read_sql(query, con=engine)

In [5]:
print("Data loaded from MySQL with shape:", df.shape)
print(df.head())

Data loaded from MySQL with shape: (410, 9)
   id                   timestamp   region device_id customer_id  latency_ms  \
0   1  2025-10-13 17:13:09.483702    South   DEV-018    CUST-004    86.94730   
1   2  2025-10-10 03:13:09.483587     East   DEV-001    CUST-001    49.30650   
2   3  2025-10-10 09:13:09.483807  Central   DEV-003    CUST-001    16.98860   
3   4  2025-10-11 13:13:09.483599  Unknown   DEV-017    CUST-013    51.68820   
4   5  2025-10-08 10:13:09.483585    South   DEV-005    CUST-006     6.87705   

   uptime_percent  packet_loss_rate  bandwidth_usage_mb  
0         92.7019          0.567788             891.004  
1         95.9159          0.000000            1757.910  
2         99.0012          3.997050            1710.020  
3         89.3120          0.000000            1228.900  
4         94.0393          0.000000            1945.050  


In [6]:
# ----------------
# 1. Overall KPIs
# ----------------

avg_latency = df['latency_ms'].mean()
avg_uptime = df['uptime_percent'].mean()
avg_packet_loss = df['packet_loss_rate'].mean()
total_bandwidth = df['bandwidth_usage_mb'].sum()

print("\nOverall etwork KPIs:")
print(f"Average Latency (ms): {avg_latency:.2f} ms")
print(f"Average Uptime (%): {avg_uptime:.2f}%")
print(f"Average Packet Loss Rate (%): {avg_packet_loss:.2f}%")
print(f"Total Bandwidth Usage (MB): {total_bandwidth:.2f} MB\n")


Overall etwork KPIs:
Average Latency (ms): 49.31 ms
Average Uptime (%): 90.01%
Average Packet Loss Rate (%): 2.44%
Total Bandwidth Usage (MB): 443397.32 MB



In [7]:
# ----------------
# 2. KPIs by Region
# ----------------

region_summary = (
    df.groupby('region')
    .agg({
        'latency_ms': 'mean',
        'uptime_percent': 'mean',
        'packet_loss_rate': 'mean',
        'bandwidth_usage_mb': 'sum'
    })
    .reset_index()
    )

print("Regional performance summary:")
print(region_summary, "\n")

Regional performance summary:
    region  latency_ms  uptime_percent  packet_loss_rate  bandwidth_usage_mb
0  Central   45.557799       89.642271          2.981780           68640.316
1     East   51.980831       90.095977          2.430067           88418.724
2    North   51.506147       89.668595          2.477021           91339.552
3    South   47.229428       90.337595          2.092380           98727.254
4  Unknown   47.681650       94.490167          2.045373           15209.274
5     West   50.061275       89.511564          2.371019           81062.200 



In [8]:
# ----------------
# 3. Detecy Underperforming Devices
# ----------------

poor_regions = region_summary[
    (region_summary["uptime_percent"] < 90) | (region_summary["packet_loss_rate"] > 1)
]

if not poor_regions.empty:
    print("Regions with underperforming metrics:")
    print(poor_regions)
else:
    print("All regions are performing within acceptable thresholds.")

Regions with underperforming metrics:
    region  latency_ms  uptime_percent  packet_loss_rate  bandwidth_usage_mb
0  Central   45.557799       89.642271          2.981780           68640.316
1     East   51.980831       90.095977          2.430067           88418.724
2    North   51.506147       89.668595          2.477021           91339.552
3    South   47.229428       90.337595          2.092380           98727.254
4  Unknown   47.681650       94.490167          2.045373           15209.274
5     West   50.061275       89.511564          2.371019           81062.200
